# PaddleHub实战——使用语义预训练模型ERNIE 进行多标签文本分类

# 使用百度试题数据 95分类

试题知识点标注

## 注意

本项目代码需要使用GPU环境来运行:

本项目将演示，如何使用PaddleHub语义预训练模型ERNIE对自定义数据集完成文本分类。

In [ ]:
# 安装PaddleHub
!pip install --upgrade paddlehub -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 204kB 8.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 36.9MB/s eta 0:00:01
     |████████████████████████████████| 174kB 42.5MB/s eta 0:00:01
     |████████████████████████████████| 81kB 35.0MB/s eta 0:00:01
     |████████████████████████████████| 184kB 44.3MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 48.6MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 43.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 21.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 26.2MB/s eta 0:00:01
     |████████████████████████████████| 8.0MB 43.1MB/s eta 0:00:01
     |████████████████████████████████| 102kB 40.6MB/s ta 0:00:01
     |████████████████████████████████| 81kB 36.8MB/s eta 0:00:01
     |████████████████████████████████| 2.7MB 43.8MB/s eta 0:00:01
     |████████████████████████████████| 573kB 42.6MB/s eta 0:00:01
     |██

## PART I. 加载自定义数据集

开课吧&amp;后厂理工学院 百度NLP项目2：百度试题数据集多标签文本分类

# 数据说明
原始数据集为`高中`下`地理`,`历史`,`生物`,`政治`四门学科数据，每个学科下各包含第一层知识点，如`历史`下分为`近代史`,`现代史`,`古代史`。  
原始数据示例： 

> [题目]  
我国经济体制改革首先在农村展开。率先实行包产到组、包产到户的农业生产责任制的省份是（    ）  
①四川        ②广东        ③安徽       ④湖北A. ①③B. ①④C. ②④D. ②③题型: 单选题|难度: 简单|使用次数: 0|纠错复制收藏到空间加入选题篮查看答案解析答案：A解析：本题主要考察的是对知识的识记能力，比较容易。根据所学知识可知，在四川和安徽，率先实行包产到组、包产到户的农业生产责任制，故①③正确；②④不是。所以答案选A。知识点：  
[知识点：]  
经济体制改革,中国的振兴

对数据处理：
- 将数据的[知识点：]作为数据的第四层标签，显然不同数据的第四层标签数量不一致
- 仅保留题目作为数据特征，删除[题型]及[答案解析]

加载自定义数据集，用户仅需要继承HubDataset类，替换数据集存放地址即可。 下面代码示例展示如何将自定义数据集加载进PaddleHub使用。

具体详情可参考 加载[自定义数据集](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub%E9%80%82%E9%85%8D%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E5%AE%8C%E6%88%90FineTune)

In [ ]:
import os
import pandas as pd
import paddlehub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# 读取数据

In [ ]:
DATA_OUTPUT_DIR='/home/aistudio/data/data22677/'
data_path=os.path.join(DATA_OUTPUT_DIR,'baidu_95.csv')
df=pd.read_csv(data_path,header=None, names=["labels", "content"], dtype=str)

In [ ]:
df.head()

,labels,content
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜从土壤中吸收的氮元素可以用来合成（）A.淀粉和纤维素B.葡萄糖和DNAC.核酸和蛋白质D...
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列有关生物体内信息传递的叙述，正确的是（）A下丘脑分泌的促甲状腺激素释放激素，可作用于甲状...
2,高中 生物 生物技术实践 生物工程技术,从自然菌样筛选较理想生产菌种的一般步骤是：采集菌样→富集培养→纯种分离→性能测定．1.不同微...
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前，精子载体法逐渐成为具有诱惑力的制备转基因动物方法之一，该方法以精子作为外源基因的载体，...
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人想乘普通飞机在一年中连续过两次生日，你认为应穿越（）A赤道B两级C本初子午线D国际日期变更线


# 预处理

In [ ]:
df['labels']=df['labels'].apply(lambda x:x.split())
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['labels'])
y=[' '.join([str(j) for j in i]) for i in y.tolist()]
df['labels']=y

classes_df=pd.DataFrame(mlb.classes_)

In [ ]:
classes_df

,0
0,“重农抑商”政策
1,不完全显性
2,与细胞分裂有关的细胞器
3,中央官制——三公九卿制
4,中心体的结构和功能
5,人体免疫系统在维持稳态中的作用
6,人体水盐平衡调节
7,人体的体温调节
8,人口与城市
9,人口增长与人口问题


In [ ]:
df.head()

,labels,content
0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,菠菜从土壤中吸收的氮元素可以用来合成（）A.淀粉和纤维素B.葡萄糖和DNAC.核酸和蛋白质D...
1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,下列有关生物体内信息传递的叙述，正确的是（）A下丘脑分泌的促甲状腺激素释放激素，可作用于甲状...
2,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,从自然菌样筛选较理想生产菌种的一般步骤是：采集菌样→富集培养→纯种分离→性能测定．1.不同微...
3,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,目前，精子载体法逐渐成为具有诱惑力的制备转基因动物方法之一，该方法以精子作为外源基因的载体，...
4,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,某人想乘普通飞机在一年中连续过两次生日，你认为应穿越（）A赤道B两级C本初子午线D国际日期变更线


In [ ]:
# 训练数据集划分
train_df, test_df = train_test_split(df, test_size=0.8, random_state=42)
# 测试和验证集划分
test_df, dev_df = train_test_split(test_df, test_size=0.5, random_state=42)

# 保存数据
train_df.to_csv(os.path.join(DATA_OUTPUT_DIR, 'train.tsv'),index=None,header=None)
dev_df.to_csv(os.path.join(DATA_OUTPUT_DIR, 'dev.tsv'),index=None,header=None)
test_df.to_csv(os.path.join(DATA_OUTPUT_DIR, 'test.tsv'),index=None,header=None)

# 保存类别标签
classes_df.to_csv(os.path.join(DATA_OUTPUT_DIR,'classes.csv'),index=None,header=None)

In [ ]:
train_df.head()

,labels,content
12580,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,12月24日当地时间19∶00时，英格兰足球超级联赛的一场比赛将在伦敦开赛。香港李先生要看这...
21349,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,下图①表示两条非同源染色体，则①→②的变化在遗传学上称为（）A.易位B.缺失C.倒位D.重复
19700,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 ...,γ­氨基丁酸和某种局部麻醉药在神经兴奋传递过程中的作用机理如图1所示。此种局麻药单独使用时不...
12225,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 ...,古典经济学家斯密认为，每个人在追求仅仅是他个人的利益时，有一只看不见的手引导他去促进一种目标...
1051,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,用加酶洗衣粉洗涤衣物时，下列说法错误的是()A.可用加酶洗衣粉洗涤毛料衣服B.一般先用热水泡...


# 数据加载

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import namedtuple
import codecs
import os
import csv

from paddlehub.dataset.dataset import InputExample, BaseDataset
from paddlehub.common.downloader import default_downloader
from paddlehub.common.dir import DATA_HOME
from paddlehub.common.logger import logger


class Baidu95(BaseDataset):
    """
    ChnSentiCorp (by Tan Songbo at ICT of Chinese Academy of Sciences, and for
    opinion mining)
    """

    def __init__(self):
        self.dataset_dir = DATA_OUTPUT_DIR
        if not os.path.exists(self.dataset_dir):
            logger.info("Dataset not exists.".format(self.dataset_dir))
        else:
            logger.info("Dataset {} already cached.".format(self.dataset_dir))

        self._load_train_examples()
        self._load_test_examples()
        self._load_dev_examples()

    def _load_train_examples(self):
        self.train_file = os.path.join(self.dataset_dir, "train.tsv")
        self.train_examples = self._read_tsv(self.train_file)

    def _load_dev_examples(self):
        self.dev_file = os.path.join(self.dataset_dir, "dev.tsv")
        self.dev_examples = self._read_tsv(self.dev_file)

    def _load_test_examples(self):
        self.test_file = os.path.join(self.dataset_dir, "test.tsv")
        self.test_examples = self._read_tsv(self.test_file)

    def get_train_examples(self):
        return self.train_examples

    def get_dev_examples(self):
        return self.dev_examples

    def get_test_examples(self):
        return self.test_examples

    def get_labels(self):
        return pd.read_csv(os.path.join(DATA_OUTPUT_DIR,'classes.csv'),names=['labels'])['labels'].tolist()

    @property
    def num_labels(self):
        """
        Return the number of labels in the dataset.
        """
        return len(self.get_labels())

    def _read_tsv(self, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with codecs.open(input_file, "r", encoding="UTF-8") as f:
            reader = csv.reader(f, delimiter=",", quotechar=quotechar)
            examples = []
            seq_id = 0
            header = next(reader)  # skip header
            for line in reader:
                example = InputExample(
                    guid=seq_id, label=[int(i) for i in line[0].split(' ')], text_a=line[1])
                seq_id += 1
                examples.append(example)

            return examples

In [ ]:
dataset = Baidu95()

[2020-03-01 20:41:33,528] [    INFO] - Dataset /home/aistudio/data/data22677/ already cached.


In [ ]:
e=dataset.get_val_examples()[0]
e.label

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0]

In [ ]:
!hub install ernie_tiny

[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmp7la3935_/ernie_tiny
[==================================================] 100.00%


In [ ]:
# 更换name参数即可无缝切换BERT中文模型, 代码示例如下
max_seq_len=256

module = hub.Module(name="bert_chinese_L-12_H-768_A-12") # (name="bert_chinese_L-12_H-768_A-12")
inputs, outputs, program = module.context(trainable=True, max_seq_len=max_seq_len)

[2020-03-01 21:03:58,751] [    INFO] - Installing bert_chinese_L-12_H-768_A-12 module


[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmpiuiyy62_/bert_chinese_L-12_H-768_A-12
[==================================================] 100.00%


[2020-03-01 21:04:15,356] [    INFO] - Successfully installed bert_chinese_L-12_H-768_A-12-1.0.0
[2020-03-01 21:04:15,849] [    INFO] - Set maximum sequence length of input tensor to 256
[2020-03-01 21:04:15,851] [    INFO] - The shape of input tensor[input_ids] set to [-1, 256, 1]
[2020-03-01 21:04:15,852] [    INFO] - The shape of input tensor[position_ids] set to [-1, 256, 1]
[2020-03-01 21:04:15,852] [    INFO] - The shape of input tensor[segment_ids] set to [-1, 256, 1]
[2020-03-01 21:04:15,853] [    INFO] - The shape of input tensor[input_mask] set to [-1, 256, 1]
[2020-03-01 21:04:15,853] [    INFO] - 199 pretrained paramaters loaded by PaddleHub


如果想尝试其他语义模型（如ernie_tiny, R等），只需要更换Module中的`name`参数即可.

   模型名                           | PaddleHub Module
---------------------------------- | :------:
ERNIE, Chinese                     | `hub.Module(name='ernie')`
ERNIE 2.0 Tiny, Chinese            | `hub.Module(name='ernie_tiny')`
ERNIE 2.0 Base, English            | `hub.Module(name='ernie_v2_eng_base')`
ERNIE 2.0 Large, English           | `hub.Module(name='ernie_v2_eng_large')`
RoBERTa-Large, Chinese             | `hub.Module(name='roberta_wwm_ext_chinese_L-24_H-1024_A-16')`
RoBERTa-Base, Chinese              | `hub.Module(name='roberta_wwm_ext_chinese_L-12_H-768_A-12')`
BERT-Base, Uncased                 | `hub.Module(name='bert_uncased_L-12_H-768_A-12')`
BERT-Large, Uncased                | `hub.Module(name='bert_uncased_L-24_H-1024_A-16')`
BERT-Base, Cased                   | `hub.Module(name='bert_cased_L-12_H-768_A-12')`
BERT-Large, Cased                  | `hub.Module(name='bert_cased_L-24_H-1024_A-16')`
BERT-Base, Multilingual Cased      | `hub.Module(nane='bert_multi_cased_L-12_H-768_A-12')`
BERT-Base, Chinese                 | `hub.Module(name='bert_chinese_L-12_H-768_A-12')`

In [ ]:
dataset = Baidu95()
reader = hub.reader.MultiLabelClassifyReader(
    dataset=dataset,
    vocab_path=module.get_vocab_path(),
    max_seq_len=max_seq_len,
    use_task_id=False)
metrics_choices = ['acc','f1']

[2020-03-01 21:04:21,209] [    INFO] - Dataset /home/aistudio/data/data22677/ already cached.
[2020-03-01 21:04:22,256] [    INFO] - Dataset label map = {'“重农抑商”政策': 0, '不完全显性': 1, '与细胞分裂有关的细胞器': 2, '中央官制——三公九卿制': 3, '中心体的结构和功能': 4, '人体免疫系统在维持稳态中的作用': 5, '人体水盐平衡调节': 6, '人体的体温调节': 7, '人口与城市': 8, '人口增长与人口问题': 9, '人口迁移与人口流动': 10, '人工授精、试管婴儿等生殖技术': 11, '伴性遗传': 12, '体液免疫的概念和过程': 13, '免疫系统的功能': 14, '免疫系统的组成': 15, '公民道德与伦理常识': 16, '兴奋在神经元之间的传递': 17, '兴奋在神经纤维上的传导': 18, '内环境的稳态': 19, '内质网的结构和功能': 20, '农业区位因素': 21, '减数分裂与有丝分裂的比较': 22, '减数分裂的概念': 23, '分子与细胞': 24, '劳动就业与守法经营': 25, '历史': 26, '古代史': 27, '器官移植': 28, '地球与地图': 29, '地球所处的宇宙环境': 30, '地球的内部圈层结构及特点': 31, '地球的外部圈层结构及特点': 32, '地球运动的地理意义': 33, '地球运动的基本形式': 34, '地理': 35, '垄断组织的出现': 36, '培养基与无菌技术': 37, '基因工程的原理及技术': 38, '基因工程的概念': 39, '基因的分离规律的实质及应用': 40, '基因的自由组合规律的实质及应用': 41, '复等位基因': 42, '夏商两代的政治制度': 43, '太阳对地球的影响': 44, '宇宙中的地球': 45, '工业区位因素': 46, '拉马克的进化学说': 47, '政治': 48, '文艺的春天': 49, '核糖体的结构和功能': 50, '海峡两岸关系的发展': 51, '液泡的结构和功能': 52, '清末民主革

# 优化器设置

In [ ]:
strategy = hub.AdamWeightDecayStrategy(
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_proportion=0.0,
    lr_scheduler="linear_decay",
)

config = hub.RunConfig(use_cuda=True, num_epoch=5, batch_size=32, strategy=strategy)

[2020-03-01 21:04:26,907] [    INFO] - Checkpoint dir: ckpt_20200301210426


# 运行模型

In [ ]:
# Define a classfication finetune task by PaddleHub's API
pooled_output = outputs["pooled_output"]
feed_list = [
        inputs["input_ids"].name, 
        inputs["position_ids"].name,
        inputs["segment_ids"].name,
        inputs["input_mask"].name
    ]
    
multi_label_cls_task = hub.MultiLabelClassifierTask(
    data_reader=reader,
    feature=pooled_output,
    feed_list=feed_list,
    num_classes=dataset.num_labels,
    config=config)

In [33]:
# Finetune and evaluate by PaddleHub's API
# will finish training, evaluation, testing, save model automatically
multi_label_cls_task.finetune_and_eval()

[2020-03-01 21:06:25,337] [    INFO] - PaddleHub finetune start
[2020-03-01 21:06:30,689] [   TRAIN] - step 20 / 705: loss=9.77301 auc=0.49437 auc_“重农抑商”政策=0.57934 auc_不完全显性=0.40746 auc_与细胞分裂有关的细胞器=0.42031 auc_中央官制——三公九卿制=0.55427 auc_中心体的结构和功能=0.46659 auc_人体免疫系统在维持稳态中的作用=0.60898 auc_人体水盐平衡调节=0.60219 auc_人体的体温调节=0.49780 auc_人口与城市=0.38321 auc_人口增长与人口问题=0.42704 auc_人口迁移与人口流动=0.36719 auc_人工授精、试管婴儿等生殖技术=0.53354 auc_伴性遗传=0.47908 auc_体液免疫的概念和过程=0.58285 auc_免疫系统的功能=0.61374 auc_免疫系统的组成=0.58770 auc_公民道德与伦理常识=0.46795 auc_兴奋在神经元之间的传递=0.49388 auc_兴奋在神经纤维上的传导=0.56420 auc_内环境的稳态=0.55572 auc_内质网的结构和功能=0.49728 auc_农业区位因素=0.72576 auc_减数分裂与有丝分裂的比较=0.49517 auc_减数分裂的概念=0.48851 auc_分子与细胞=0.52601 auc_劳动就业与守法经营=0.46257 auc_历史=0.51521 auc_古代史=0.49807 auc_器官移植=0.60180 auc_地球与地图=0.25410 auc_地球所处的宇宙环境=0.51257 auc_地球的内部圈层结构及特点=0.49784 auc_地球的外部圈层结构及特点=0.54857 auc_地球运动的地理意义=0.51094 auc_地球运动的基本形式=0.51656 auc_地理=0.54446 auc_垄断组织的出现=0.45287 auc_培养基与无菌技术=0.51673 auc_基因工程的原理及技术=0.53358 auc_基因工程的概念=0.59918 auc_基因的分离规律的实质

# 预测

In [ ]:
import numpy as np

# 预测数据
data = [[d.text_a, d.text_b] for d in dataset.get_test_examples()]
# 预测标签
test_label = np.array([d.label for d in dataset.get_test_examples()])
# 预测
run_states = multi_label_cls_task.predict(data)

In [ ]:
def inverse_predict_array(batch_result):
    return np.argmax(batch_result, axis=2).T

In [ ]:
results = [run_state.run_results for run_state in run_states]
predict_label=np.concatenate([inverse_predict_array(batch_result) for batch_result in results])

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
print('f1 micro:{}'.format(f1_score(test_label,predict_label,average='micro')))
print('f1 samples:{}'.format(f1_score(test_label,predict_label,average='samples')))
print('f1 macro:{}'.format(f1_score(test_label,predict_label,average='macro')))

In [ ]:
mlb.inverse_transform(predict_label)[2:5]

In [ ]:
mlb.inverse_transform(test_label)[2:5]

## PART II. PaddleHub一键加载ERNIE


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_1.png" hspace='10'/> <br />
</p>


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_2.png" hspace='10'/> <br />
</p>

In [ ]:
import paddlehub as hub
module = hub.Module(name="ernie")

如果想尝试其他语义模型（如ernie_tiny, R等），只需要更换Module中的`name`参数即可.

   模型名                           | PaddleHub Module
---------------------------------- | :------:
ERNIE, Chinese                     | `hub.Module(name='ernie')`
ERNIE 2.0 Tiny, Chinese            | `hub.Module(name='ernie_tiny')`
ERNIE 2.0 Base, English            | `hub.Module(name='ernie_v2_eng_base')`
ERNIE 2.0 Large, English           | `hub.Module(name='ernie_v2_eng_large')`
RoBERTa-Large, Chinese             | `hub.Module(name='roberta_wwm_ext_chinese_L-24_H-1024_A-16')`
RoBERTa-Base, Chinese              | `hub.Module(name='roberta_wwm_ext_chinese_L-12_H-768_A-12')`
BERT-Base, Uncased                 | `hub.Module(name='bert_uncased_L-12_H-768_A-12')`
BERT-Large, Uncased                | `hub.Module(name='bert_uncased_L-24_H-1024_A-16')`
BERT-Base, Cased                   | `hub.Module(name='bert_cased_L-12_H-768_A-12')`
BERT-Large, Cased                  | `hub.Module(name='bert_cased_L-24_H-1024_A-16')`
BERT-Base, Multilingual Cased      | `hub.Module(nane='bert_multi_cased_L-12_H-768_A-12')`
BERT-Base, Chinese                 | `hub.Module(name='bert_chinese_L-12_H-768_A-12')`


## PART III. 构建Reader

接着生成一个文本分类的reader，reader负责将dataset的数据进行预处理，首先对文本进行切词，接着以特定格式组织并输入给模型进行训练。

`ClassifyReader`的参数有以下三个：
* `dataset`: 传入PaddleHub Dataset;
* `vocab_path`: 传入ERNIE/BERT模型对应的词表文件路径;
* `max_seq_len`: ERNIE模型的最大序列长度，若序列长度不足，会通过padding方式补到max_seq_len, 若序列长度大于该值，则会以截断方式让序列长度为max_seq_len;
* `sp_model_path`: 传入 ERNIE tiny的subword切分模型路径;
* `word_dict_path`: 传入 ERNIE tiny的词语切分模型路径;

<center> <img width="600px" src="https://ai-studio-static-online.cdn.bcebos.com/c1a201c80acc46708cedc1c73614b0531b716c0299474859bf423ce0a5160170" /> </center>

In [ ]:
reader = hub.reader.ClassifyReader(
    dataset=dataset,
    vocab_path=module.get_vocab_path(),
    max_seq_len=128)

## PART IV、选择Fine-Tune优化策略
适用于ERNIE/BERT这类Transformer模型的迁移优化策略为`AdamWeightDecayStrategy`。详情请查看[Strategy](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub-API:-Strategy)。

`AdamWeightDecayStrategy`的参数：
> * `learning_rate`: 最大学习率
> * `lr_scheduler`: 有`linear_decay`和`noam_decay`两种衰减策略可选
> * `warmup_proprotion`: 训练预热的比例，若设置为0.1, 则会在前10%的训练step中学习率逐步提升到`learning_rate`
> * `weight_decay`: 权重衰减，类似模型正则项策略，避免模型overfitting
> * `optimizer_name`: 优化器名称


<center> <img width="900px" src="https://ai-studio-static-online.cdn.bcebos.com/e7d3b9df642d443a8bdab1af0f3de93c0fa111ac5d3c4c9c88b2aa12f92dba64" /> </center>


In [ ]:
strategy = hub.AdamWeightDecayStrategy(
    weight_decay=0.01,
    warmup_proportion=0.1,
    learning_rate=5e-5,
    lr_scheduler="linear_decay",
    optimizer_name="adam")

## PART V. 选择运行配置

在进行Finetune前，我们可以设置一些运行时的配置，例如如下代码中的配置，表示：

> `use_cuda`：设置为False表示使用CPU进行训练。如果您本机支持GPU，且安装的是GPU版本的PaddlePaddle，我们建议您将这个选项设置为True；
>
> `num_epoch`：Finetune时遍历训练集的次数，；
>
> `batch_size`：每次训练的时候，给模型输入的每批数据大小为16，模型训练时能够并行处理批数据，因此batch_size越大，训练的效率越高，但是同时带来了内存的负荷，过大的batch_size可能导致内存不足而无法训练，因此选择一个合适的batch_size是很重要的一步；
>
> `log_interval`：每隔10 step打印一次训练日志；
>
> `eval_interval`：每隔50 step在验证集上进行一次性能评估；
>
> `checkpoint_dir`：训练的参数和数据的保存目录；
>
> `strategy`：Fine-tune策略；
>
> `use_data_parallel`: 设置为False表示单卡训练；设置为True表示多卡训练
>
> `use_pyreader`: 设置为False表示不使用py_reader读取数据；设置为True表示使用py_reader读取数据

更多运行配置，请查看[RunConfig](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub-API:-RunConfig)

In [ ]:
config = hub.RunConfig(
    use_cuda=True,
    num_epoch=1,
    checkpoint_dir="hub_ernie_text_cls_demo",
    batch_size=32,
    log_interval=10,
    eval_interval=50,
    use_pyreader=False,
    use_data_parallel=False,
    strategy=strategy)

## PART VI. 组建Finetune Task

有了合适的预训练模型和准备要迁移的数据集后，我们开始组建一个Task。

1. 获取module的上下文环境，包括输入和输出的变量，以及Paddle Program；
2. 从输出变量中找到用于情感分类的文本特征pooled_output；
3. 在pooled_output后面接入一个全连接层，生成Task；

`TextClassifierTask`的参数有：

> `data_reader`：读取数据的reader；
>
> `config`: 运行配置；
>
> `feature`：从预训练提取的特征；
>
> `feed_list`：program需要输入的变量；
>
> `num_classes`：数据集的类别数量；
> 
> `metric_choic`：任务评估指标，默认为"acc"。metrics_choices支持训练过程中同时评估多个指标，作为最佳模型的判断依据，例如["matthews", "acc"]，"matthews"将作为主指标，为最佳模型的判断依据；


<center> <img width="600px" src="https://user-images.githubusercontent.com/48793257/69508060-97c0c180-0f6f-11ea-9886-00300da42683.png" /> </center>


In [ ]:
inputs, outputs, program = module.context(
    trainable=True, max_seq_len=128)

# Use "pooled_output" for classification tasks on an entire sentence.
pooled_output = outputs["pooled_output"]

feed_list = [
    inputs["input_ids"].name,
    inputs["position_ids"].name,
    inputs["segment_ids"].name,
    inputs["input_mask"].name,
]

cls_task = hub.TextClassifierTask(
        data_reader=reader,
        feature=pooled_output,
        feed_list=feed_list,
        num_classes=dataset.num_labels,
        config=config,
        metrics_choices=["acc"])

**NOTE：** Reader参数max_seq_len、moduel的context接口参数max_seq_len三者应该保持一致，最大序列长度`max_seq_len`是可以调整的参数，建议值128，根据任务文本长度不同可以调整该值，但最大不超过512。

### Part VII. 开始Finetune
我们选择finetune_and_eval接口来进行模型训练，这个接口在finetune的过程中，会周期性的进行模型效果的评估，以便我们了解整个训练过程的性能变化。

In [ ]:
run_states = cls_task.finetune_and_eval()

## PART VIII. 使用模型进行预测

当Finetune完成后，我们使用模型来进行预测，整个预测流程大致可以分为以下几步：
1. 构建网络
2. 生成预测数据的Reader
3. 切换到预测的Program
4. 加载预训练好的参数
5. 运行Program进行预测

<center> <img width="1024px" src="https://ai-studio-static-online.cdn.bcebos.com/320a7c13fcb044d3b45cb4720012bd26dbc10c283aa747dab43f72d1bad18643" /> </center>

> 预测代码如下：

In [ ]:
import numpy as np
    
inv_label_map = {val: key for key, val in reader.label_map.items()}

# Data to be prdicted
data = [[d.text_a, d.text_b] for d in dataset.get_test_examples()[:3]]

index = 0
run_states = cls_task.predict(data=data)

results = [run_state.run_results for run_state in run_states]
for batch_result in results:
    # get predict index
    batch_result = np.argmax(batch_result, axis=2)[0]
    for result in batch_result:
        print("%s\tpredict=%s" % (data[index][0], inv_label_map[result]))
        index += 1

**总的来说，PaddleHub完成迁移学习过程只需下图所展示的6步即可完成。**


<center> <img width="1024px" src="https://ai-studio-static-online.cdn.bcebos.com/34f2e41444bd4dd38fc9f318a551538f5cb5157075cb4c99a7f7f56306fbd37e" /> </center>